# Automatic memory management

Reduces programmer burden, integral to OOP languages, part of mainstream computing.  
Brought about by safe pointers (programs may not access arbitrary addresses, compiler can identify and provide to garbage collector all pointers)

Issue with efficiency, identifying garbage is hard and expensive

# Garbage collection fundamentals

1. Allocation (same as explicit memory management): Free list, bump allocation
2. Identification: Tracing (traces pointers to identify what is reachable), reference counting (programmer keeps track of how many objects are using a pointer)
3. Reclamation: taking back the heap

Garbage is any object that program will never reference. Any object that the program cannot reach is theoretically garbage.

# Reference counting

Count the number of references to each object, if the reference count is zero, the object is garbage, does not work for cycles.

# Tracing

Trace reachability from program roots, marks the objects reachable from the roots. Objects nor marked are unreachable

# Garbage collection algorithms

Evaluation: space efficiency, efficiency of allocator, time to collect garbage

# Reclamation

2 broad approaches:
1. Non-copying: uses free list allocation and reclamation, only way for explicit memory management
2. Copying: generally uses bump pointer allocation, en masse reclamation

# Mark Sweep

Free lists organized by size.  
Allocation: grab a pointer to free space off the free list. No more memory of the right size triggers a collection.  
Mark phase: Find the unreachable objects  
Sweep phase: put unreachable ones on the free list

Good space efficiency and incremental object reclamation, poor allocation time and locality of contemporaneously allocated objects.   
Allocator has a slow allocation time, is space efficient, poor locality

# Mark Compact

Mark compact is similar to mark sweep, except there is one more step: copy all the objects remaining in the heap to one end of the heap. This allows the mark compact to use bump pointer rather than free list.

Allocator is fast and exhibits good locality (objects are next to each other now), garbage collector is now slower than mark sweep, but is space efficient. Overall performance is worse than mark sweep, but you get the locality advantage.

# Semi-space

Fast bump pointer allocation, requires copying collection, cannot incrementally reclaim memory, must free en masse. Reserves half of the heap to copy into in case all objects are reachable. 

Originally objects are allocated in the left part of the heap. Once this half has been fully allocated, the trace algorithm is run (traces reachability from root), and all objects are copied into the right side of the heap.  
Mark phase copies object when collection first encounters it, installs forwarding pointers, which tells where the object is in case another pointer still points to the old place, performs transitive closuers, so that the right side of the heap has reachable objects only, collects the left side of the heap en masse, starts allocating again on the left side

Fast allocation, locality of objects, just wasted space. Best locality, garbage collection is fast as well.

# One big heap?

Takes too long to trace the whole heap, heap contains lots of long lived objects, why collect them over and over, divide up the heap into increments and collect one at a time.  
Organize heap into young and old, collect young objects preferetially based on generational hypothesis: young objects die more quickly than older ones.  

# Generational Heap

Divide heap into two spaces, allocate young space, and when that fills up, collect it and copy into the old space. When the old space fills up, collect both spaces.

# I/O Interfacing

Hardware components communicate using the system bus.  
A bus allows the device to communicate with the CPU. A port consists of 4 registers: status (whether the device is busy or ready or error), control (command to perform), data-in (data being sent from the device to the CPU), data-out (data being sent from the CPU to the device), controller that receives commands from the system bus, translates commands, and read/writes data to the system bus, the device itself

# Devices

Characteristics:
1. Transfer unit: character or block
2. access method: sequential or random
3. Timing: Synchronous/asynchronous
4. Shared or dedicated
5. Speed
6. Operations: Input, output, or both

OS usually communicates through controller.  
places information in device registers for controller to send to device  
controller places informatino from devices in the register for the OS

# Polling

OS keeps checking until the status of I/O device is idle.  
Check rate is polling rate.  
OS sets command register  
OS sets status to command-ready. 
Controlller sets status to busy. 
Controller reads the command register and performs the command, and if it is an input operation, places the value as data-in.  
Assuming the operation succeeded, the controller changes status to idle.   
OS observes change and reads the data if it is an input operation

# Interrupts

Rather than having the CPU check if a device is available, have the device interrupt the CPU when I/O is done.  
On I/O interrupt, determine which deice caused the interrupt, and retrieve data if it is an input operation, start the next operation

# DMA (direct memory access)

Device uses a more sophisticated controller that can write to memory.  
CPU tells DMA controller lcoation of the source and destination of transfer.   
DMA controller operates the bus and interrupts the CPU when entire transfer is complete.   
DMA and CPU compete for memory bus, but is still better performance

OS provides high level interfaces to devices, standard interfaces provided for related devices, device intricacies encapsulated in device drivers, new devices supported by providing a device driver with device.

# Putting it together, typical read calls

1. User process requests a read from device
2. OS checks if data is in buffer, if not, OS tells device driver to perform input, device driver tells DMA controller what to do and blocks itself, DMA controller transfers data, and when it has been complete, interrupts the CPU
3. OS transfers data to the user process and places the process in the ready queue
4. When the process gets the CPU it begins execution following the system call